<a href="https://colab.research.google.com/github/leonammeta8154/Estela_IA/blob/main/Estela_teste_Agentes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip -q install google-genai

In [2]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [3]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [4]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 Para saber a data da próxima Imersão IA com Google Gemini da Alura, o melhor é verificar diretamente no site da Alura:

*   **Site da Alura:** Procure na página de cursos ou na seção de imersões e bootcamps. Eles costumam anunciar as próximas datas por lá.

Além disso, você pode:

*   **Seguir a Alura nas redes sociais:** Eles costumam divulgar as novidades por lá.
*   **Assinar a newsletter da Alura:** Assim, você recebe as informações diretamente no seu e-mail.

Assim que tiver informações mais concretas, posso te ajudar!

In [5]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu de 12 a 16 de maio de 2025. As inscrições encerraram no dia 11 de maio de 2025.


In [6]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['Alura Imersão IA com Google Gemini próxima edição']
Páginas utilizadas na resposta: thallesbenicio.com.br, alemdoclicktech.com.br



In [7]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [8]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService # memoria interna desse agente.
from google.adk.tools import google_search # ferramenta do google
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [9]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [10]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [11]:
##########################################
# --- Agente 1: Buscador de Notícias --- #
##########################################

def agente_buscador(topico, data_de_hoje):
  buscador = Agent(
      name ="agente_buscador",
      model="gemini-2.0-flash",
      description="agente de buscar informações no google",
      tools=[google_search],
      instruction="""
      Você é um assistente de pesquisa. A sua tarefa é usar a ferramenta de busca do (google_search) para recuperar as últimas notícias de lançamentos muito relevantes sobre o tópico abaixo.
      Foque em no máximo 5 lançamentos relevantes, com base na quantidade e entusiasmo das notícias sobre ele.
      Se um tema tiver poucas notícias ou reações entusiasmadas, é posspivel que ele não seja tão relevante aasim
      esse lançamenbto relevantes devem ser atuais, de no máximo um mês antes da data de hoje.
      """
  )

  entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
  # Executa o agente
  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos_buscados

In [12]:
################################################
# --- Agente 2: Planejador de posts --- #
################################################
def agente_planejador(topico, lancamentos_buscados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""

        Você é um planejador de conteúdo, especialista em redes sociais. Com base na lista de lanlamentos mais recentes e revelantes buscador, você deve:
        usar a ferramenta de busca do google (google_search) para criar um plano sobre quais são os pontos mais relevantes que poderíamos abordar em um post sobre cada um deles. Você também pode usar o (google_search) para encontrar mais
        informações sobre os temas e aprofundar.
        Ao final, você irá escolher o tema mais relevante entre eles com base nas suas pesquisas e retornar esse tema, seus pontos mais relevantes, e um plano com os asuntos
        a serem abordados no post que será escrito posteriormente.
        """,
        description="Agente que planeja posts",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico}\nLançamentos buscados: {lancamentos_buscados}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [13]:
######################################
# --- Agente 3: Redator do Post --- #
######################################
def agente_redator(topico, plano_de_post):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Alura, a maior escola online de tecnologia do Brasil.
            Utilize o tema fornecido no plano de post e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para Instagram sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de post: {plano_de_post}"
    # Executa o agente
    rascunho = call_agent(redator, entrada_do_agente_redator)
    return rascunho

In [14]:
##########################################
# --- Agente 4: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, rascunho_gerado):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em posts para redes sociais, com foco no Instagram.
            Por ter um público jovem, entre 18 e 30 anos, use um tom de escrita adequado.
            Revise o rascunho de post de Instagram abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Se o rascunho estiver bom, responda apenas 'O rascunho está ótimo e pronto para publicar!'.
            Caso haja problemas, aponte-os e sugira melhorias.
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {rascunho_gerado}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [15]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("Você esqueceu de digitar o tópico!")
else:
    print(f"Maravilha! Vamos então criar o post sobre novidades em {topico}")

    lancamentos_buscados = agente_buscador(topico, data_de_hoje)
    print("\n--- 📝 Resultado do Agente 1 (Buscador) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

    plano_de_post = agente_planejador(topico, lancamentos_buscados)
    print("\n--- 📝 Resultado do Agente 2 (Planejador) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")

    rascunho_de_post = agente_redator(topico, plano_de_post)
    print("\n--- 📝 Resultado do Agente 3 (Redator) ---\n")
    display(to_markdown(rascunho_de_post))
    print("--------------------------------------------------------------")

    post_final = agente_revisor(topico, rascunho_de_post)
    print("\n--- 📝 Resultado do Agente 4 (Revisor) ---\n")
    display(to_markdown(post_final))
    print("--------------------------------------------------------------")

🚀 Iniciando o Sistema de Criação de Posts para Instagram com 4 Agentes 🚀
❓ Por favor, digite o TÓPICO sobre o qual você quer criar o post de tendências: academia
Maravilha! Vamos então criar o post sobre novidades em academia

--- 📝 Resultado do Agente 1 (Buscador) ---



> Para fornecer as informações mais recentes sobre lançamentos relevantes na área de academia, realizarei algumas buscas no Google. Meu objetivo é encontrar novidades que tenham gerado bastante atenção e entusiasmo no último mês.
> 
> 
> Com base nas minhas buscas, aqui estão alguns lançamentos e tendências relevantes na área de academia para 2025:
> 
> 1.  **Tecnologia Vestível Avançada:** Smartwatches e rastreadores de atividade física estão se tornando mais sofisticados, oferecendo monitoramento em tempo real e integração com treinos. A expectativa é que, em 2025, esses dispositivos sejam integrados à inteligência artificial, permitindo análises detalhadas do desempenho físico e oferecendo sugestões personalizadas de treino.
> 
> 2.  **Aplicativos Móveis Imersivos:** Combinando realidade aumentada (AR) e realidade virtual (VR), os aplicativos de treino prometem enriquecer a experiência dos alunos, proporcionando interatividade e personalização diretamente no smartphone.
> 
> 3.  **Equipamentos Fitness de Recuperação:** Equipamentos voltados para recuperação e bem-estar, como cadeiras de massagem, máquinas de crioterapia e saunas, estão ganhando espaço nas academias. A popularidade dos alongamentos também continuará a crescer, à medida que os clientes buscam maneiras mais eficazes de prevenir lesões e melhorar a performance.
> 
> 4.  **Treinamento Híbrido:** O futuro das academias será híbrido, combinando treinos presenciais com plataformas online. Essa abordagem oferece flexibilidade e conveniência, atendendo a diferentes perfis de alunos.
> 
> 5.  **Programas de Condicionamento Físico para Idosos:** Com o aumento da longevidade, a atividade física para idosos ganha destaque. Em 2025, espera-se um crescimento significativo de programas voltados para a terceira idade, focados em melhorar a qualidade de vida e prevenir doenças crônicas.
> 
> Além dessas tendências, outras inovações notáveis incluem o uso de inteligência artificial em equipamentos, permitindo que se adaptem ao treino em tempo real, e a gamificação dos treinos, com a inclusão de desafios e recompensas para engajar os alunos. A conscientização sobre a longevidade e a importância de cuidados integrados, que englobam a alimentação e a saúde mental, também são destaques para 2025.


--------------------------------------------------------------

--- 📝 Resultado do Agente 2 (Planejador) ---



> Okay, com base nas informações que você forneceu e em minhas próprias buscas, vou criar um plano de conteúdo para cada um dos tópicos listados, identificando os pontos mais relevantes e elaborando sugestões de abordagem para um post de mídia social.
> 
> **1. Tecnologia Vestível Avançada:**
> 
> *   **Pontos Relevantes:**
>     *   Integração de IA para análise detalhada e personalizada do desempenho.
>     *   Monitoramento em tempo real (frequência cardíaca, sono, etc.).
>     *   Sugestões de treino adaptadas ao usuário.
>     *   Conectividade com outros dispositivos e aplicativos de saúde.
> *   **Plano de Conteúdo:**
>     *   Título chamativo: "Seu Personal Trainer no Pulso: A Revolução da IA nos Smartwatches Fitness!"
>     *   Imagem/Vídeo: Um vídeo curto mostrando um smartwatch em ação, exibindo dados e personalizações de treino.
>     *   Tópicos a serem abordados:
>         *   O que são wearables e como eles evoluíram.
>         *   Como a IA está transformando o monitoramento fitness.
>         *   Exemplos de smartwatches com funcionalidades avançadas.
>         *   Benefícios de usar wearables para otimizar seus treinos.
>         *   O futuro da tecnologia vestível e o que esperar em 2025.
>     *   Call to Action: "Descubra como a tecnologia vestível pode turbinar seus resultados! Link na bio para saber mais."
> 
> **2. Aplicativos Móveis Imersivos (AR/VR):**
> 
> *   **Pontos Relevantes:**
>     *   Experiência de treino interativa e personalizada.
>     *   Realidade aumentada para sobrepor informações ao mundo real.
>     *   Realidade virtual para simular ambientes de treino.
>     *   Gamificação e desafios para aumentar o engajamento.
> *   **Plano de Conteúdo:**
>     *   Título chamativo: "Academia na Palma da Mão: A Revolução AR/VR nos Seus Treinos!"
>     *   Imagem/Vídeo: Um vídeo curto mostrando um app de AR/VR em uso, com alguém treinando em um ambiente virtual ou com informações sobrepostas.
>     *   Tópicos a serem abordados:
>         *   O que são aplicativos de treino AR/VR e como eles funcionam.
>         *   Como a imersão aumenta a motivação e o engajamento.
>         *   Exemplos de aplicativos populares com funcionalidades AR/VR.
>         *   Benefícios de treinar com realidade aumentada e virtual.
>         *   O futuro dos aplicativos de treino imersivos.
>     *   Call to Action: "Transforme seus treinos com a realidade aumentada! Experimente os melhores apps de AR/VR: link na bio."
> 
> **3. Equipamentos Fitness de Recuperação:**
> 
> *   **Pontos Relevantes:**
>     *   Cadeiras de massagem, máquinas de crioterapia e saunas para recuperação muscular.
>     *   Foco na prevenção de lesões e melhora da performance.
>     *   Aumento da popularidade dos alongamentos e técnicas de relaxamento.
> *   **Plano de Conteúdo:**
>     *   Título chamativo: "Recupere Como um Atleta: As Novas Tendências em Equipamentos de Recuperação!"
>     *   Imagem/Vídeo: Fotos ou vídeos de equipamentos de recuperação em uma academia, ou pessoas alongando.
>     *   Tópicos a serem abordados:
>         *   A importância da recuperação muscular para o desempenho.
>         *   Como equipamentos como cadeiras de massagem e crioterapia ajudam na recuperação.
>         *   Benefícios dos alongamentos e técnicas de relaxamento.
>         *   Como incorporar a recuperação na sua rotina de treino.
>     *   Call to Action: "Leve a sério a recuperação muscular e previna lesões! Agende uma sessão de recuperação na nossa academia: link na bio."
> 
> **4. Treinamento Híbrido:**
> 
> *   **Pontos Relevantes:**
>     *   Combinação de treinos presenciais e plataformas online.
>     *   Flexibilidade e conveniência para diferentes perfis de alunos.
>     *   Acesso a treinos personalizados e acompanhamento remoto.
> *   **Plano de Conteúdo:**
>     *   Título chamativo: "O Futuro do Fitness é Híbrido: Treine Onde e Quando Quiser!"
>     *   Imagem/Vídeo: Um vídeo curto mostrando pessoas treinando tanto na academia quanto em casa, com acompanhamento online.
>     *   Tópicos a serem abordados:
>         *   O que é treinamento híbrido e como ele funciona.
>         *   Benefícios de combinar treinos presenciais e online.
>         *   Como encontrar um programa de treinamento híbrido adequado.
>         *   Dicas para aproveitar ao máximo o treinamento híbrido.
>     *   Call to Action: "Descubra a liberdade do treinamento híbrido! Saiba mais sobre nossos programas online e presenciais: link na bio."
> 
> **5. Programas de Condicionamento Físico para Idosos:**
> 
> *   **Pontos Relevantes:**
>     *   Foco na melhoria da qualidade de vida e prevenção de doenças crônicas.
>     *   Exercícios adaptados às necessidades e limitações da terceira idade.
>     *   Crescimento significativo da demanda por programas específicos.
> *   **Plano de Conteúdo:**
>     *   Título chamativo: "Viva Mais e Melhor: Programas de Condicionamento Físico para a Terceira Idade!"
>     *   Imagem/Vídeo: Fotos ou vídeos de idosos se exercitando em um ambiente seguro e supervisionado.
>     *   Tópicos a serem abordados:
>         *   A importância da atividade física para a saúde dos idosos.
>         *   Benefícios de programas de condicionamento físico específicos.
>         *   Exemplos de exercícios seguros e eficazes para a terceira idade.
>         *   Como encontrar um programa adequado e começar a se exercitar.
>     *   Call to Action: "Invista na sua saúde e bem-estar na terceira idade! Conheça nossos programas especiais para idosos: link na bio."
> 
> **Escolha do Tema Mais Relevante e Plano Detalhado:**
> 
> Considerando o contexto atual e as tendências para 2025, o tema mais relevante para um post de mídia social sobre academia é o **Treinamento Híbrido**. A combinação de treinos presenciais e online oferece flexibilidade e conveniência, atendendo a uma gama maior de pessoas com diferentes estilos de vida e preferências.
> 
> **Plano Detalhado para o Post sobre Treinamento Híbrido:**
> 
> *   **Título:** "O Futuro do Fitness é Híbrido: Treine Onde e Quando Quiser!" (ou um título similar, chamativo e direto).
> *   **Imagem/Vídeo:** Um vídeo curto e dinâmico mostrando pessoas treinando em diferentes cenários (academia, casa, ao ar livre), utilizando plataformas online para acompanhamento e orientação.
> *   **Conteúdo do Post:**
>     *   **Introdução:**
>         *   Uma frase impactante sobre a evolução do fitness e a necessidade de flexibilidade.
>         *   Definição concisa de treinamento híbrido: "A combinação perfeita entre treinos presenciais e online, para você alcançar seus objetivos onde e quando quiser."
>     *   **Corpo do Post:**
>         *   **Benefícios do Treinamento Híbrido:**
>             *   **Flexibilidade:** Adapte os treinos à sua rotina e compromissos.
>             *   **Conveniência:** Treine onde e quando for mais fácil para você.
>             *   **Variedade:** Experimente diferentes modalidades e estilos de treino.
>             *   **Acompanhamento Personalizado:** Tenha acesso a orientação profissional online e presencial.
>             *   **Resultados:** Maximize seus resultados com um plano de treino completo e adaptado às suas necessidades.
>         *   **Como Encontrar um Programa de Treinamento Híbrido Adequado:**
>             *   Pesquise academias e profissionais que ofereçam programas híbridos.
>             *   Verifique a qualidade do conteúdo online e o suporte oferecido.
>             *   Considere seus objetivos e preferências ao escolher um programa.
>         *   **Dicas para Aproveitar ao Máximo o Treinamento Híbrido:**
>             *   Defina metas claras e realistas.
>             *   Crie uma rotina de treino consistente.
>             *   Aproveite os recursos online e o acompanhamento profissional.
>             *   Mantenha-se motivado e engajado com a comunidade online.
>     *   **Conclusão:**
>         *   Reafirme a importância do treinamento híbrido como o futuro do fitness.
>         *   Incentive os seguidores a experimentar essa modalidade.
> *   **Call to Action:** "Descubra a liberdade do treinamento híbrido! Saiba mais sobre nossos programas online e presenciais e comece a treinar hoje mesmo: link na bio."
> *   **Hashtags:** #treinamentohibrido #fitness #academia #treinoonline #saude #bemestar #qualidadedevida #futurodofitness
> 
> Espero que este plano detalhado seja útil para a criação do seu post!
> 


--------------------------------------------------------------

--- 📝 Resultado do Agente 3 (Redator) ---



> **💪 O Futuro do Fitness é Híbrido: Treine Onde e Quando Quiser! 🚀**
> 
> Chegou a hora de revolucionar seus treinos! 🤩 Já pensou em combinar a energia da academia com a praticidade de treinar onde e quando quiser? 🤔 Com o treinamento híbrido, você tem o melhor dos dois mundos!
> 
> ✅ **Flexibilidade:** Adapte os treinos à sua rotina, sem desculpas!
> ✅ **Conveniência:** Sua academia no bolso, 24/7.
> ✅ **Resultados:** Acompanhamento online + presencial para turbinar seus objetivos!
> 
> Descubra como o treinamento híbrido pode transformar sua vida! ✨
> 
> 👉 Quer saber mais sobre nossos programas online e presenciais? Clique no link da bio e comece a treinar hoje mesmo! 😉
> 
> #treinamentohibrido #fitness #academia #saude


--------------------------------------------------------------

--- 📝 Resultado do Agente 4 (Revisor) ---



> O rascunho está bom, mas podemos fazer alguns ajustes para deixá-lo ainda mais atraente para o público do Instagram:
> 
> *   **Emojis:** Use emojis de forma mais estratégica para quebrar o texto e destacar os benefícios.
> *   **Chamada para Ação (CTA):** A CTA poderia ser mais direta e urgente.
> *   **Hashtags:** Adicione hashtags mais específicas para aumentar o alcance.
> 
> **Sugestão de revisão:**
> 
> **💪 TREINO HÍBRIDO: A solução pra quem não tem tempo! 🚀**
> 
> Chegou a hora de dar um UP nos seus treinos! 🤩 Já pensou em ter a energia da academia combinada com a liberdade de treinar onde e quando quiser? 🤔 Com o treino híbrido, você tem o melhor dos dois mundos! 🌍 + 📱
> 
> ✅ **Flexibilidade:** Sem tempo? Sem problema! 💪 Adapte o treino à sua rotina.
> ✅ **Praticidade:** Leve a academia com você! 🏋️‍♀️Seu treino 24/7, onde estiver.
> ✅ **Resultados:** Acompanhamento online + presencial = FOCO nos seus objetivos! 🎯
> 
> Descubra como o treino híbrido pode te ajudar a ter mais resultados! ✨
> 
> 👉 Quer saber mais e começar AGORA? 🚨 Clique no link da bio e bora treinar! 😉
> 
> \#treinohibrido \#fitness \#academia \#vidasaudavel \#semdesculpas \#foconotreino
> 


--------------------------------------------------------------
